<a href="https://colab.research.google.com/github/RafsanJany-44/SIgnal_Processing/blob/main/Signal_Processing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install mne

In [15]:
from glob import glob
import os
import mne
import pandas
import matplotlib.pyplot as plt

In [16]:
data_path = "/content/drive/MyDrive/Signal_Data/dataverse_files"

In [17]:
all_file_path = glob(data_path+"/*.edf")
print(len(all_file_path))

28


In [18]:
all_file_path

['/content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h05.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h06.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h04.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h08.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h09.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h11.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h02.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h07.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h03.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h01.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h10.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/s01.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/s07.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h13.edf',
 '/content/drive/MyDrive/Signal_Data/dataverse_files/h1

In [19]:
healthy_file_path = []
patient_file_path = []

for i in all_file_path:
  if str(os.path.basename(i))[0] == 'h':
    healthy_file_path.append(i)
  else:
    patient_file_path.append(i)

In [27]:
def read_data(file_path):
  data = mne.io.read_raw_edf(file_path, preload=True)
  data.set_eeg_reference()
  data.filter(l_freq=0.5,h_freq=45)
  epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
  array=epochs.get_data()
  return array

In [28]:
sample_data = read_data(healthy_file_path[0])

Extracting EDF parameters from /content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
224 matching events found
No baseline correction applied
0 projection i

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


In [30]:
sample_data.shape   #epochs,channels, length of signals

(224, 19, 1250)

In [31]:
control_epochs_array = []
patient_epochs_array = []

for i in healthy_file_path:
  control_epochs_array.append(read_data(i))

for i in patient_file_path:
  patient_epochs_array.append(read_data(i))

Extracting EDF parameters from /content/drive/MyDrive/Signal_Data/dataverse_files/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
224 matching events found
No baseline correction applied
0 projection i

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 236249  =      0.000 ...   944.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
236 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 236 events and 1250 original time points ...
0 bad epochs drop

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 232499  =      0.000 ...   929.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 232 events and 1250 original time points ...
0 bad epochs drop

...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 212499  =      0.000 ...   849.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
212 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 212 events and 1250 original time points ...
0 bad epochs drop

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 283749  =      0.000 ...  1134.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
283 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 283 events and 1250 original time points ...
0 bad epochs drop

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
